### pySpark ml word2vec

In [3]:
# 設定spark session 環境變數 spark home
# 詳細 ?findspark.init
import findspark
findspark.init('/usr/local/spark')
# 載入必要modules/
import time
from pyspark.sql import SparkSession
from pyspark.sql import functions as fn
from pyspark.ml.feature import Word2Vec, Word2VecModel

hdfs_path = "hdfs://your_ip:port"
coresMax = "4"
executorMem = "12g"

# 開啟 spark session
# pyspark session 目前需要手動設定使用到的參數
# spark.executor.memory worker 計算/cache相關記憶體大小
# spark.driver.memory 與 driver 相關的記憶體大小影響task分派，對演算速度影響大
# spark.driver.maxResultSize 設定分散資料collect回本地端的大小上限，SparkR有更改設定檔無限制，此處須設置。
ss = SparkSession.builder \
    .master( "spark://your_ip:port" ) \
    .appName( "word_to_vector" ) \
    .config( "spark.cores.max", coresMax ) \
    .config( "spark.executor.memory", executorMem ) \
    .config( "spark.driver.memory", "20g" ) \
    .config( "spark.driver.maxResultSize", "20g" ) \
    .config( "spark.rpc.message.maxSize", "512") \
    .getOrCreate() 

In [2]:
# 讀取 parquet 資料源
# 使用資料 wiki, 一些 ppt, 電商, 部落格的文章
corpus = ss.read.parquet( hdfs_path + "/your_path/*.parquet" )
corpus.printSchema()

root
 |-- source: string (nullable = true)
 |-- categroy: string (nullable = true)
 |-- url: string (nullable = true)
 |-- title: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [5]:
corpus.show(3)

+------+--------+--------------------+-------------+--------------------+
|source|categroy|                 url|        title|               words|
+------+--------+--------------------+-------------+--------------------+
|  wiki|        |https://zh.wikipe...|   浸信宣道會呂明才小學|[浸信, 宣道, 呂明, 小學, ...|
|  wiki|        |https://zh.wikipe...|        索拉庫喬山|[索拉, 庫喬, 索拉, 庫喬, ...|
|  wiki|        |https://zh.wikipe...|阿南塔山 (阿雷基帕大區)|[阿南, 塔山, 阿雷基帕, 大區...|
+------+--------+--------------------+-------------+--------------------+
only showing top 3 rows



In [6]:
# 由於是每天爬文，會有少量文章重複，因此要刪除重複的文章
corpus = corpus.dropDuplicates()
corpus.count()

2010510

In [8]:
# 設定 word2vec 參數，並進行訓練和儲存模型
# numPartitions 使用和 core 數相同，maxIter 數官方文件建議須小於等於 partition 數
# maxSentenceLength 設定為 8000，我們 corpus 的長度都超過 1000，預設數值明顯不符合需求。需要確認文章長度平均值及測試。
word2Vec = Word2Vec( vectorSize = 240, 
                     inputCol = "words", 
                     outputCol = "word2vec",
                     windowSize = 6,
                     numPartitions = 8,
                     stepSize = 0.05, 
                     maxIter = 8,
                     maxSentenceLength = 8000 )
w2v_model = word2Vec.fit( corpus )
w2v_model.write().overwrite().save( hdfs_path + "/your_path/WordToVector_model.ml" )

In [5]:
# getVectors() 方法是取得 w2v 的 embedding，其中 word 是字，vector 是該字長為 240 的 embedding
w2v_model.getVectors().show(5)

+---------+--------------------+
|     word|              vector|
+---------+--------------------+
|       凍人|[10852.51171875,-...|
|       克幸|[102047.2578125,-...|
|hoshizora|[1119.98083496093...|
|      王義芳|[1990.271484375,-...|
|       焦尼|[11.6127252578735...|
+---------+--------------------+
only showing top 5 rows



In [ ]:
# 取出 word embedding 另外儲存，供後續 sparkR 分析使用，由於 sparkR 不能針對 vector 進行分析，因此先轉為 array 在儲存
w2v_vectors = w2v_model.getVectors()
w2v_embedding = w2v_vectors.rdd.map(lambda row: (row[0], row[1].toArray().tolist()) ).toDF(schema=['word', 'embedding'])
w2v_embedding.write.parquet( path = hdfs_path + "/your_path/WordToVector_embedding.parquet", 
                             mode = "overwrite",
                             compression = "gzip" )

In [13]:
# 嘗試 word2vec 訓練成效
w2v_model.findSynonyms( "女王", 10 ).show()

+----+------------------+
|word|        similarity|
+----+------------------+
|  王子|0.6979262828826904|
|伊莉莎白|0.6587029099464417|
|瑪格麗特|0.6476824283599854|
|伊麗莎白|0.6196021437644958|
|  亨利|0.6115877032279968|
|  王后|0.6103411912918091|
|  國王|0.6059281826019287|
|  二世|0.5922608971595764|
|  瑪麗|0.5908293724060059|
|  安娜|0.5880428552627563|
+----+------------------+



In [14]:
ss.stop()

In [4]:
# 須先啟動 spark session 才能讀取模型
from pyspark.ml.feature import Word2VecModel
w2v_model = Word2VecModel.load( hdfs_path + "/your_path/WordToVector_model.ml" )